In [1]:
import os
import numpy as np
import keras
from keras import layers
import pandas as pd

from src import mnist
from src import config
from src import analysis as an

In [ ]:
def lock_keras_model(model, key):
    # Get current weights and biases from the Keras model
    # get_weights() returns a flat list: [W1, b1, W2, b2, W3, b3, W4, b4]
    keras_weights_and_biases = model.get_weights()

    # Extract only the weight matrices (kernel) from the list
    weight_matrices = [keras_weights_and_biases[i] for i in range(0, len(keras_weights_and_biases), 2)]
    bias_vectors = [keras_weights_and_biases[i] for i in range(1, len(keras_weights_and_biases), 2)]

    permutation_matrices = []
    
    # Generate permutation matrices based on the shapes of the weight matrices
    for i in range(4): # For W1, W2, W3, W4
        W = weight_matrices[i]
        
        # P_row (permutation for rows of W)
        P_row = np.eye(W.shape[0])
        np.random.seed(key[2 * i])
        np.random.shuffle(P_row)
        permutation_matrices.append(P_row)

        # P_col (permutation for columns of W)
        P_col = np.eye(W.shape[1])
        np.random.seed(key[2 * i + 1])
        np.random.shuffle(P_col)
        permutation_matrices.append(P_col)

    # Apply permutations to weights: W_new = P_row  W_old  P_col
    new_weight_matrices = []
    for i in range(4):
        W_old = weight_matrices[i]
        P_row = permutation_matrices[2 * i]
        P_col = permutation_matrices[2 * i + 1]
        
        # Perform matrix multiplications
        permuted_W = np.dot(P_row, W_old)
        permuted_W = np.dot(permuted_W, P_col)
        new_weight_matrices.append(permuted_W)

    # Reconstruct the list of weights and biases to set back into the model
    # The order must be [new_W1, b1, new_W2, b2, new_W3, b3, new_W4, b4]
    updated_keras_weights_and_biases = []
    for i in range(4):
        updated_keras_weights_and_biases.append(new_weight_matrices[i])
        updated_keras_weights_and_biases.append(bias_vectors[i])

    # Set the permuted weights back into the Keras model
    model.set_weights(updated_keras_weights_and_biases)
    print("\nModel weights have been 'locked' (permuted).")

In [ ]:
print("Loading MNIST dataset using custom MNIST class...")

try:
    mnist_data_loader = mnist.MNIST(
        config.TRAINING_IMAGES_FILEPATH, config.TRAINING_LABELS_FILEPATH,
        config.TEST_IMAGES_FILEPATH, config.TEST_LABELS_FILEPATH
    )
    x_train, y_train = mnist_data_loader.x_train, mnist_data_loader.y_train
    x_test, y_test = mnist_data_loader.x_test, mnist_data_loader.y_test
    print("MNIST dataset loaded and preprocessed using custom class.")
except FileNotFoundError as e:
    print(f"Error loading MNIST files: {e}. Please ensure the MNIST dataset files are in the correct directory.")
    print("Expected paths:")
    print(f"  Training Images: {config.TRAINING_IMAGES_FILEPATH}")
    print(f"  Training Labels: {config.TRAINING_LABELS_FILEPATH}")
    print(f"  Test Images: {config.TEST_IMAGES_FILEPATH}")
    print(f"  Test Labels: {config.TEST_LABELS_FILEPATH}")
    exit()

num_classes = 10
model = keras.Sequential(
    [
        keras.Input(shape=(784,)), # Input shape is 28*28 = 784
        layers.Dense(512, activation="relu", name="hidden_layer_1"),
        layers.Dense(56, activation="relu", name="hidden_layer_2"),
        layers.Dense(128, activation="relu", name="hidden_layer_3"),
        layers.Dense(num_classes, activation="softmax", name="output_layer"),
    ]
)


model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

print("\n--- Starting Model Training on MNIST ---")
history = model.fit(x_train, y_train, epochs=10, batch_size=128, validation_split=0.1)
print("--- Model Training Complete ---")

print("\n--- Evaluating the Model on Test Set ---")
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")
print("--- Model Evaluation Complete ---")

raw_data_df = pd.read_csv(r'data\raw\v2\pynq_1_data.csv', dtype=str)
lock_key = an.convert_binary_to_naturals(an.get_ideal_value(raw_data_df, 'PUF_Response_Value'), chunk_size=16)
print(f'locking keys = {lock_key}')
lock_keras_model(model, lock_key)

print("\n--- Exporting Weights and Biases to CSV files ---")

# Create a directory to store the CSV files if it doesnt exist
output_dir = r'models\mnist_mlp'
os.makedirs(output_dir, exist_ok=True)

for i, layer in enumerate(model.layers):
    if isinstance(layer, layers.Dense):
        weights, biases = layer.get_weights()

        # Save weights to a .npy file (lossless binary format)
        weight_filename = os.path.join(output_dir, f"mnist_mlp_w{i+1}.npy")
        np.save(weight_filename, weights)
        print(f"Saved weights for {layer.name} to {weight_filename} (shape: {weights.shape})")

        # Save biases to a .npy file
        bias_filename = os.path.join(output_dir, f"mnist_mlp_b{i+1}.npy")
        np.save(bias_filename, biases)
        print(f"Saved biases for {layer.name} to {bias_filename} (shape: {biases.shape})")

print(f"\nAll weights and biases exported to the '{output_dir}' directory.")

Loading MNIST dataset using custom MNIST class...
MNIST Data loaded and processed:
  x_train shape: (60000, 784), dtype: float32
  y_train shape: (60000, 10), dtype: float64
  x_test shape: (10000, 784), dtype: float32
  y_test shape: (10000, 10), dtype: float64
MNIST dataset loaded and preprocessed using custom class.


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ hidden_layer_1 (Dense)          │ (None, 512)            │       401,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ hidden_layer_2 (Dense)          │ (None, 56)             │        28,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ hidden_layer_3 (Dense)          │ (None, 128)            │         7,296 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output_layer (Dense)            │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 439,234 (1.68 MB)

 Trainable params: 439,234 (1.68 MB)

 Non-trainable params: 0 (0.00 B)


--- Starting Model Training on MNIST ---
Epoch 1/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8339 - loss: 0.5525 - val_accuracy: 0.9677 - val_loss: 0.1175
Epoch 2/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9664 - loss: 0.1113 - val_accuracy: 0.9763 - val_loss: 0.0795
Epoch 3/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9802 - loss: 0.0626 - val_accuracy: 0.9777 - val_loss: 0.0762
Epoch 4/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9866 - loss: 0.0420 - val_accuracy: 0.9783 - val_loss: 0.0747
Epoch 5/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9900 - loss: 0.0314 - val_accuracy: 0.9782 - val_loss: 0.0748
Epoch 6/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9929 - loss: 0.0227 - val_accuracy: 0.9792 - val_loss: 0.0752
Epoch 7/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9942 - loss: 0.0194 - val_accuracy: 0.9818 - val_loss: 0.0692
Epoch 8/10
422/422 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy